## Week6: Digital Humanities
#### Group: Jennifer, Vedasri
https://www.kdnuggets.com/2020/07/spam-filter-python-naive-bayes-scratch.html
i am usingabove link

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [7]:
train = pd.read_csv('corpus/sms_spam.train.csv', delimiter=",")
print("Traing shape: ",train.shape)

test = pd.read_csv('corpus/sms_spam.test.csv', delimiter=",")
print("Test shape  : ",test.shape)
test.head(5)

Traing shape:  (5199, 2)
Test shape  :  (361, 2)


,type,text
0,ham,happened here while you were adventuring
1,ham,"Ask g or iouri, I've told the story like ten t..."
2,ham,"Sorry, I'll call later"
3,spam,Great News! Call FREEFONE 08006344447 to claim...
4,ham,Dont know supports srt i thnk. I think ps3 can...


#### Class-wise training data

In [8]:
train['type'].value_counts(normalize=True)

ham     0.866513
spam    0.133487
Name: type, dtype: float64

#### Class-wise training data

In [10]:
test['type'].value_counts(normalize=True)

ham     0.853186
spam    0.146814
Name: type, dtype: float64

#### Cleaning the data

In [76]:
train_punctuation = train.copy()
train_punctuation['text'] = train['text'].str.replace('\W', ' ')

train_lower = train_punctuation.copy()
train_lower['text'] = train_punctuation['text'].str.lower()
train_lower.head(5)


,type,text
0,ham,happened here while you were adventuring
1,ham,ask g or iouri i ve told the story like ten t...
2,ham,sorry i ll call later
3,spam,great news call freefone 08006344447 to claim...
4,ham,dont know supports srt i thnk i think ps3 can...


In [77]:


test_punctuation = test.copy()
test_punctuation['text'] = test['text'].str.replace('\W', ' ')

test_lower = test_punctuation.copy()
test_lower['text'] = test_punctuation['text'].str.lower()
test_lower.head(5)




,type,text
0,ham,happened here while you were adventuring
1,ham,ask g or iouri i ve told the story like ten t...
2,ham,sorry i ll call later
3,spam,great news call freefone 08006344447 to claim...
4,ham,dont know supports srt i thnk i think ps3 can...


### Trying to train on 30 most occurance in ham and 30 spam features. But need to update top 30 keys to main train_lower['text'] column with  type ham & spam

In [13]:
train_lower_only_word_tokens = train_lower.copy()

for i,s in enumerate(train_lower['text']):
    only_word_tokens = re.findall("[a-z]+", s, re.I)
    train_lower_only_word_tokens['text'][i] =  (" ").join(only_word_tokens)

train_lower_only_word_tokens.head(5)

,type,text
0,ham,hope you are having a good week just checking in
1,ham,k give back my thanks
2,ham,am also doing in cbe only but have to pay
3,spam,complimentary star ibiza holiday or cash needs...
4,spam,okmail dear dave this is your final notice to ...


In [15]:
# removing stop_words
stop_words_1 = np.loadtxt('corpus/stopwords.txt', dtype='str')
train_remove_stop_words = train_lower_only_word_tokens.copy()

for index,sms in enumerate(train_lower_only_word_tokens['text']):
    token_without_sw = [word for word in sms.split(" ") if word not in stop_words_1]
    train_remove_stop_words['text'][index] = (" ").join(token_without_sw)

train_remove_stop_words.head(5) 

,type,text
0,ham,hope good week checking
1,ham,k give back thanks
2,ham,also cbe pay
3,spam,complimentary star ibiza holiday cash needs ur...
4,spam,okmail dear dave final notice collect tenerife...


In [16]:
stop_words_2 = np.loadtxt('corpus/StopwordSMART.txt', dtype='str')
train_remove_stop_words_2 = train_remove_stop_words.copy()

for index, sms in enumerate(train_remove_stop_words['text']):
    token_without_sw = [word for word in sms.split(" ") if  word not in stop_words_2]
    train_remove_stop_words_2['text'][index] = (" ").join(token_without_sw) 

train_remove_stop_words_2.head(5)

,type,text
0,ham,hope good week checking
1,ham,give back
2,ham,cbe pay
3,spam,complimentary star ibiza holiday cash urgent c...
4,spam,okmail dear dave final notice collect tenerife...


In [18]:
ham_tokens = train_remove_stop_words_2.loc[train_remove_stop_words_2['type'] == 'ham']
print("Total no. of hams in training set: ",len(ham_tokens))
spam_tokens = train_remove_stop_words_2.loc[train_remove_stop_words_2['type'] == 'spam']
print("Total no. of Spams in training set:",len(spam_tokens))

Total no. of hams in training set:  4505
Total no. of Spams in training set: 694


In [21]:
from collections import Counter
words_all = []

for i, words in enumerate(ham_tokens['text']):
    total_words = words.split(" ")
    for w in total_words: 
        words_all.append(w)
        
words_dict = Counter(words_all)
dict_sorted = {k: v for k, v in sorted(words_dict.items(), key=lambda item: item[1], reverse=True)}

#print(dict_sorted)
words_ham_30 = list(dict_sorted.keys())[:30]
print(words_ham_30)

['ur', 'call', 'good', 'day', 'love', 'time', 'home', 'lor', 'da', 'dont', 'today', 'back', 'send', 'pls', 'night', 'hey', 'wat', 'dear', 'happy', 'hope', '', 'great', 'give', 'work', 'yeah', 'make', 'im', 'morning', 'phone', 'tomorrow']


In [23]:
words_all = []

for i, words in enumerate(spam_tokens['text']):
    total_words = words.split(" ")
    for w in total_words: 
        words_all.append(w)
        
words_dict = Counter(words_all)
dict_sorted = {k: v for k, v in sorted(words_dict.items(), key=lambda item: item[1], reverse=True)}
words_spam_30 = list(dict_sorted.keys())[:30]
print(words_spam_30)

['call', 'free', 'txt', 'ur', 'stop', 'mobile', 'text', 'claim', 'reply', 'www', 'prize', 'uk', 'send', 'cash', 'win', 'nokia', 'urgent', 'contact', 'msg', 'tone', 'week', 'service', 'box', 'guaranteed', 'customer', 'ppm', 'mins', 'phone', 'cs', 'chat']


In [25]:
#list(set(words_spam_30).intersection(words_ham_30))

### Train on total sms_text data set in my system unable to split spam ham rows to spam_messages, ham_messages

In [26]:
# Creating the vocabulary

#train_split = train_lower.copy()
#train_split['text'] = train_lower['text'].str.split()
#print(len(train_lower))
# vocabulary = []

# for index, sms in enumerate(train_lower['text']):
#     total_words = [vocabulary.append(w) for w in sms.split(" ")]
        
# vocabulary = list(set(vocabulary))
# len(vocabulary)

train_lower['text'] = train_lower['text'].str.split()

vocabulary = []
for sms in train_lower['text']:
    for word in sms:
        vocabulary.append(word)

vocabulary = list(set(vocabulary))
len(vocabulary)

8384

In [38]:
# Creating word counts per test

top_30_features = words_spam_30+ words_ham_30
word_counts_per_sms = {unique_word: [0] * len(train_lower['text']) for unique_word in top_30_features}

#print(train_lower['text'])

for index, sms in enumerate(train_lower['text']):
    for word in sms:
        if word in top_30_features:
            word_counts_per_sms[word][index] += 1


In [39]:
# Transformation of training set

word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,call,free,txt,ur,stop,mobile,text,claim,reply,www,...,hope,,great,give,work,yeah,make,im,morning,tomorrow
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
# Concatinating Label, text to word-counts


training_set_clean = pd.concat([train_lower['type'], word_counts], axis=1)
training_set_clean['Words'] = train_lower['text']
training_set_clean.head(5)



,type,call,free,txt,ur,stop,mobile,text,claim,reply,...,,great,give,work,yeah,make,im,morning,tomorrow,Words
0,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[hope, you, are, having, a, good, week, just, ..."
1,ham,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,"[k, give, back, my, thanks]"
2,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[am, also, doing, in, cbe, only, but, have, to..."
3,spam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[complimentary, 4, star, ibiza, holiday, or, 1..."
4,spam,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[okmail, dear, dave, this, is, your, final, no..."


In [70]:
# Isolating spam and ham messages first
# idx = training_set_clean.to_frame()
# training_set_clean.loc[idx.iloc[:,0]]
spam_messages = training_set_clean[training_set_clean['type'] == 'spam']
ham_messages = training_set_clean[training_set_clean['type'] == 'ham']
spam_messages.head(5)

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# # N_Spam
n_words_per_spam_message = spam_messages['Words'].apply(len)
n_spam = n_words_per_spam_message.sum()

# # N_Ham
n_words_per_ham_message = ham_messages['Words'].apply(len)
n_ham = n_words_per_ham_message.sum()

# # N_Vocabulary
n_vocabulary = len(top_30_features)

# # Laplace smoothing
alpha = 1

In [72]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in top_30_features}
parameters_ham = {unique_word:0 for unique_word in top_30_features}

# Calculate parameters
for word in top_30_features:
    n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
    
    #print( word, n_word_given_spam )
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam

    n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

In [73]:
import re

def classify(message):
    '''
    message: a string
    '''

    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
             p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham: 
             p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')


In [74]:
def classify_test_set(message):
    '''
    message: a string
    '''

    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [78]:
test_lower['predicted'] = test_lower['text'].apply(classify_test_set)
test_lower.head()

,type,text,predicted
0,ham,happened here while you were adventuring,ham
1,ham,ask g or iouri i ve told the story like ten t...,ham
2,ham,sorry i ll call later,ham
3,spam,great news call freefone 08006344447 to claim...,spam
4,ham,dont know supports srt i thnk i think ps3 can...,ham


In [85]:
correct = 0
total = test_lower.shape[0]

for row in test_lower.iterrows():
    row = row[1]
    if row['type'] == row['predicted']:
        correct += 1

print('Correct   : {}/{}'.format(correct, total))
print('Incorrect : {}/{}'.format(total-correct, total))
print('Accuracy  : {:.2f}%'.format(100*correct/total))

Correct   : 344/361
Incorrect : 17/361
Accuracy  : 95.29%
